In [6]:
import string
from lxml import html
import requests
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import pandas as pd
import os
from datetime import datetime
import glob

In [3]:
#search for local restaurants and extract total number of page results
def_url = 'https://www.yelp.com/search?find_desc=indian+vegetarian+restaurant&find_loc=Santa+Clara,+CA'
rest_dict = {}
page = requests.get(def_url)
tree = html.fromstring(page.text)
tot_page = int(tree.xpath('//div[@class="page-of-pages arrange_unit arrange_unit--fill"]/text()')[0].strip().split()[-1])
tot_page

7

In [5]:
#Test if the retaurant is indian
page_per_rest = requests.get('https://www.yelp.com/biz/komala-vilas-sunnyvale'.split('?')[0])
tree_per_rest = html.fromstring(page_per_rest.text)
category_per_rest = tree_per_rest.xpath('//span[@class="category-str-list"]/text()')
title = tree_per_rest.xpath('//meta[@itemprop="name"]')[1].get("content")

category_per_rest


['\n                    ',
 ',\n                    ',
 ',\n                    ',
 '\n    ',
 '\n                            Indian, Vegetarian\n                        ',
 '\n                            Indian, Vegetarian, Asian Fusion\n                        ',
 '\n                            Indian, Vegetarian\n                        ',
 '\n                            Indian, Vegetarian\n                        ',
 '\n                            Indian\n                        ',
 '\n                            Indian, Vegetarian\n                        ',
 '\n                            Indian, Vegetarian, Vegan\n                        ',
 '\n                            Indian, Vegetarian\n                        ',
 '\n                            Indian\n                        ',
 '\n                            Indian\n                        ',
 '\n                    ',
 ',\n                    ',
 ',\n                    ',
 '\n    ']

In [45]:
#Fetch restaurant names, rating, and review count
def_url = 'https://www.yelp.com/search?find_desc=Indian+Vegetarian+Restaurant&find_loc=Santa+Clara,+CA'
rest_dict = {}
page = requests.get(def_url)
tree = html.fromstring(page.text)
tot_page = int(tree.xpath('//div[@class="page-of-pages arrange_unit arrange_unit--fill"]/text()')[0].strip().split()[-1])
#tot_page = 7
page_order = range(0, ((tot_page-1)*30+1), 30)


for o in page_order:
  page = requests.get(def_url + ("&start=%s" % o))
  tree = html.fromstring(page.text)
  tmp_cnt = tree.xpath('//span[@class="review-count rating-qualifier"]/text()')
  
  rev_link = []
  rev_cnt = []
  
  for i in range(len(tmp_cnt)):
    tmp_link = 'https://www.yelp.com/' + tree.xpath('//a[@data-analytics-label="biz-name"]')[i].get("href")
    if 'adredir' in tmp_link:
      #the link address is muddled and doesnt open properly if it contains the text 'adredir', hence ignore that restaurant
      tmp_link= 'none'
    elif 'ananda-bhavan-cupertino' in tmp_link:
      #ananda-bhavan-cupertino is a unclaimed business, hence there is no rating. As a result the script wrongly picks the next rating for this business.
      #Best is to handle unclaimed business and mark it 'none' automatically, due to lack of time, I am doing it manually here for this business.
      tmp_link= 'none'
    else:
      #Test if the retaurant is indian and pick only them for this round of sims
      page_per_rest = requests.get(tmp_link)
      tree_per_rest = html.fromstring(page_per_rest.text)
      category_per_rest = tree_per_rest.xpath('//span[@class="category-str-list"]/text()')
      it_is_indian = 0
    
    
      for str in category_per_rest:
        if 'Indian' in str:
            it_is_indian = 1
        if it_is_indian == 0:  
            tmp_link = 'none'
    
    rev_link.append(tmp_link)
    
    if tmp_link == 'none':
        rev_cnt.append(0)
    else:    
        page_per_rest = requests.get(tmp_link)
        tree_per_rest = html.fromstring(page_per_rest.text)
        review_cnt_rest = int(tree_per_rest.xpath('//span[@itemprop="reviewCount"]/text()')[0])
        rev_cnt.append(review_cnt_rest)
    
  if 'review_cnt' in rest_dict:
    rest_dict['review_cnt'] += rev_cnt
    rest_dict['review_link'] += rev_link
  else:
    rest_dict['review_cnt'] = rev_cnt
    rest_dict['review_link'] = rev_link
    
    
#rest_dict

In [88]:
df_rest = pd.DataFrame(rest_dict)
df_rest['review_cnt'] = df_rest['review_cnt'].astype(int)


df_select = df_rest[df_rest.review_link != 'none'][df_rest.review_cnt>10].copy()

st_url = []
df_select.to_csv('All_Ind_Rest_SC.csv')
for lnk in df_select.review_link:
  st_url.append(lnk)
len(list(set(st_url)))
#rest_dict

16

In [99]:
#Get resurant title. Skip this function, as you may want to extract the name with city name directly from the rest html link string
#Extracting restaurant title will later create issue, as it is not much of use later. Anyways you have the rest html link str
def get_title(rest_link):
    page = requests.get(rest_link)
    tree = html.fromstring(page.text)
    # This will extract the name of the business
    title = tree.xpath('//meta[@itemprop="name"]')[1].get("content")
    return ''.join(title.strip().split()) + '.csv'


df2=df_select.copy()
df2['review_link']=df2['review_link'].apply(lambda x: x.split('?')[0])
df2['file_name']=df2['review_link'].apply(get_title)
#Store all the rest, review_cnt, link
#df2.to_csv('All_Ind_Rest_SC.csv')

,review_cnt,review_link,file_name
11,1085,https://www.yelp.com//biz/merit-vegan-cuisine-...,MeritVeganCuisine.csv
15,1317,https://www.yelp.com//biz/ulavacharu-sunnyvale-2,UlavacharU.csv
20,1355,https://www.yelp.com//biz/kabab-and-currys-san...,KababandCurry’s.csv
26,1381,https://www.yelp.com//biz/madras-cafe-sunnyvale,MadrasCafe.csv
27,830,https://www.yelp.com//biz/chaats-and-currys-su...,ChaatsAndCurrys.csv
38,936,https://www.yelp.com//biz/ananda-bhavan-sunnyvale,AnandaBhavan.csv
39,1454,https://www.yelp.com//biz/naan-n-masala-milpitas,NaanNMasala.csv
42,839,https://www.yelp.com//biz/punjab-cafe-san-jose,PunjabCafe.csv
78,1161,https://www.yelp.com//biz/walia-ethiopian-cuis...,WaliaEthiopianCuisine.csv
83,957,https://www.yelp.com//biz/saravanaa-bhavan-sun...,SaravanaaBhavan.csv


In [24]:
#obtain current review cnt for each restaurants from our already extracted restaurant list (All_Ind_Rest_SC2.csv)
df1=pd.read_csv('All_Ind_Rest_SC2.csv')
df2=df1.copy()
df3=df2.copy()
review_dict = {}
for ur in df2['review_link']:
    page = requests.get(ur.split('?')[0])
    tree = html.fromstring(page.text)
    # This will extract the name of the business
    title = tree.xpath('//meta[@itemprop="name"]')[1].get("content")
    city = tree.xpath('//span[@itemprop="addressLocality"]/text()')[0]
    review_cnt = int(tree.xpath('//span[@itemprop="reviewCount"]/text()')[0])
    review_dict[ur]=review_cnt

In [98]:
#adjust the data frame with latest review count 
def adjust_rev_cnt(x):
    return review_dict[x.review_link]

#df3['review_cnt']=df3.apply(adjust_rev_cnt,axis=1)
#df3.columns=['rest_cnt','review_cnt','review_link']
#df3.to_csv("All_Ind_Rest_SC3.csv")

In [220]:
#gather all the restaurant urls to be used next to scrap them for their reviews.
start_urls = []
cnt = 0
for lnk in df3.review_link:
    cnt += 1
    if lnk != 'none':
        start_urls.append(lnk.split('?')[0])


start_rest_cnt=47   
url_and_rest_cnt=zip(start_urls,df3['rest_cnt'])[start_rest_cnt-1:]

zip(start_urls,df3['rest_cnt'])[start_rest_cnt-1:47]
#url_and_rest_cnt

[('https://www.yelp.com//biz/bharat-bazar-food-court-sunnyvale', 47)]

In [221]:
# Yelp unique url endings for each restaurant
#restaurants = ['taco-bamba-falls-church','taco-bamba-vienna']
#restaurants = ['bombay-chaat-sunnyvale-2']
#restaurants = ['komala-vilas-sunnyvale']
#start_urls = ['http://www.yelp.com/biz/%s' % s for s in restaurants]
#start_urls = ['https://www.yelp.com//biz/mangoes-indian-cuisine-santa-clara-2']
#url_and_rest_cnt=zip(start_urls,df3['rest_cnt'])[start_rest_cnt-1:1]

#for each restaurant url scrap their respective yelp websites
for ur,rest_num in url_and_rest_cnt:
    review_dict = {}
    page = requests.get(ur)
    tree = html.fromstring(page.text)
    # This will extract the name of the business
    title = tree.xpath('//meta[@itemprop="name"]')[1].get("content")
    strip_url = ur.split('?')[0].split('/')[-1]
    city = tree.xpath('//span[@itemprop="addressLocality"]/text()')[0]
    review_cnt = int(tree.xpath('//span[@itemprop="reviewCount"]/text()')[0])
#    review_cnt = 10
    page_order = range(0, (review_cnt+1), 20)

    for o in page_order:
        page = requests.get(ur + ("?start=%s" % o))
        tree = html.fromstring(page.text)

        # This will create a list of reviews/date/author
        user_id = tree.xpath('//a[@class="user-display-name js-analytics-click"]')
        user_friendcnt = tree.xpath('//li[contains(@class,"friend-count")]/b/text()')
        user_revcnt = tree.xpath('//li[contains(@class,"review-count")]/b/text()')
        reviews = tree.xpath('//p[@itemprop="description"]/text()')
        reviews_date = tree.xpath('//meta[@itemprop="datePublished"]')
        reviews_author = tree.xpath('//meta[@itemprop="author"]')
        #There is always one extra ratingValue attribute in a yelp page (one for each reviewer + one for restaurant) 
        #So the very first value extracted is for the restaurant as a whole, so when using the extracted rating list
        #shift this list by one position to get correct rating for the reviewers.
        reviews_rating = tree.xpath('//meta[@itemprop="ratingValue"]')
        
        if reviews: # check if there is no review
        
            mod_reviews = []
            mod_reviews_date = []
            mod_reviews_author = []
            mod_reviews_rating = []
            mod_user_friendcnt = []
            mod_user_revcnt = []
            mod_user_id = []
            
            for i in range(len(reviews)):
                mod_reviews_author.append(reviews_author[i].get("content"))
                mod_reviews_date.append(reviews_date[i].get("content"))
                mod_reviews_rating.append(float(reviews_rating[i+1].get("content")))
                mod_reviews.append(reviews[i])
                mod_user_friendcnt.append(user_friendcnt[i])
                mod_user_revcnt.append(user_revcnt[i])
                for id in user_id:
                    if reviews_author[i].get("content") in id.text:
                        mod_user_id.append(str(id.get("href").split("=")[1]))
                        break
                
            rest_city = (title,city)
            # This will add the reviews to a dictionary
            if 'review' in review_dict:
                review_dict['review'] += mod_reviews
                review_dict['date'] += mod_reviews_date
                review_dict['author'] += mod_reviews_author
                review_dict['rating'] += mod_reviews_rating
                review_dict['friendcnt'] += mod_user_friendcnt
                review_dict['revcnt'] += mod_user_revcnt
                review_dict['userid'] += mod_user_id
            else:
                review_dict['review'] = mod_reviews
                review_dict['date'] = mod_reviews_date
                review_dict['author'] = mod_reviews_author
                review_dict['rating'] = mod_reviews_rating
                review_dict['friendcnt'] = mod_user_friendcnt
                review_dict['revcnt'] = mod_user_revcnt
                review_dict['userid'] = mod_user_id
            
    df = pd.DataFrame(review_dict)
#    rest_csv=''.join(title.strip().split()) + '.csv'
    rest_csv = str(rest_num) + '_' + strip_url + '.csv'
    #store individual restaurant reviews as csv to be combined into one later for data analysis
    df.to_csv('new_Restaurants/' + rest_csv)
#   files.download(rest_csv)